In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import sys

sys.path.append("..")

In [ ]:
from itertools import product

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.spatial import distance_matrix

In [ ]:
from sva import utils
from sva.truth.uv import truth_uv
from sva.value import default_asymmetric_value_function
from sva.experiments import UVData, UVExperiment

Set some plotting defaults.

In [ ]:
utils.set_defaults()

# UV-vis experiment results

In [ ]:
df = pd.read_csv("../sva/truth/uv_data.csv")
df = df.drop_duplicates(subset=["NCit", "pH", "HA"])
X = df[["NCit", "pH", "HA"]].to_numpy()
Y = df.iloc[:, 4:].to_numpy()
grid = np.array([float(xx) for xx in df.columns.tolist()[4:]])

# Anomaly detection

## LocalOutlierFactor

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
lof = LocalOutlierFactor(novelty=False)
lof.fit_predict(X)
U = lof.negative_outlier_factor_ * -1
U_lof = (U - U.min()) / (U.max() - U.min())

## Covariance

In [ ]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
cov = EllipticEnvelope().fit(X)
U = cov.score_samples(X)
U_cov = (U - U.min()) / (U.max() - U.min())

# Plots

Borrowing the clusters from the original analysis.

In [ ]:
low_value_cluster_1 = np.array([[2, -16, 2], [1, -16, 2]])
low_value_cluster_2 = np.array([[6, -16, 16], [7, -16, 14], [5.25, -16, 14]])
low_value_cluster_3 = np.array([[12, 16, 11], [11, 16, 11]])
low_value_cluster_4 = np.array([[1, 16, 2], [2, 16, 2]])

In [ ]:
highest_value_point = X[np.argmax(U), :]
distances_to_highest_value_point = distance_matrix(highest_value_point.reshape(-1, 3), X).squeeze()
argsorted = np.argsort(distances_to_highest_value_point)

In [ ]:
n_closest_points = 10
high_value_cluster = X[argsorted, :][:n_closest_points, :]

In [ ]:
def get_indexes_by_cluster(cluster, X=X):
    indexes = []
    for point in cluster:
        where = np.all(X == point, axis=1)
        where = np.where(where)[0].item()
        indexes.append(where)
    return np.array(indexes)

In [ ]:
low_value_spectra = [
    Y[get_indexes_by_cluster(low_value_cluster_1), :],
    Y[get_indexes_by_cluster(low_value_cluster_2), :],
    Y[get_indexes_by_cluster(low_value_cluster_3), :],
    Y[get_indexes_by_cluster(low_value_cluster_4), :],
]

In [ ]:
low_value_value = [
    U[get_indexes_by_cluster(low_value_cluster_1)],
    U[get_indexes_by_cluster(low_value_cluster_2)],
    U[get_indexes_by_cluster(low_value_cluster_3)],
    U[get_indexes_by_cluster(low_value_cluster_4)],
]

In [ ]:
high_value_spectra = Y[get_indexes_by_cluster(high_value_cluster), :]
high_value_value = U[get_indexes_by_cluster(high_value_cluster)]

In [ ]:
prop_cycle = plt.rcParams["axes.prop_cycle"]
colors = prop_cycle.by_key()["color"]

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(5, 10), subplot_kw={"projection": "3d"}, tight_layout=True)
axes = axes.ravel()

for title, U, ax in zip(["Local Outlier Factor", "Elliptic Envelope"], [U_lof, U_cov], axes):
    ax.scatter(*low_value_cluster_1.T, color=colors[0], alpha=1, marker="o", s=50)
    ax.scatter(*low_value_cluster_2.T, color=colors[1], alpha=1, marker="o", s=50)
    ax.scatter(*low_value_cluster_3.T, color=colors[2], alpha=1, marker="o", s=50)
    ax.scatter(*low_value_cluster_4.T, color=colors[3], alpha=1, marker="o", s=50)
    # ax.scatter(*high_value_cluster.T, color="black", alpha=1, marker="o", s=50)
    ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=U, alpha=0.9, marker="o", s=20)

    ax.set_box_aspect((np.ptp(X[:, 0]), np.ptp(X[:, 1]), np.ptp(X[:, 2])))
    ax.view_init(40, 225)
    ax.set_xlabel("Volume NaCit [$\mu$L]")
    ax.set_ylabel("Volume OH$^-$ [$\mu$L]", labelpad=20)
    ax.set_zlabel("Volume HAuCl$_4$ [$\mu$L]")

    ax.set_zticks([2, 6, 10, 14])
    ax.set_xticks([2, 6, 10, 14])

    ax.set_title(title)

# plt.show()
fig.savefig("uv_anomaly_det.pdf", dpi=300, bbox_inches="tight", pad_inches=0.5)